# Assigment 3

In [1]:
import numpy as np
import igl
import meshplot as mp
import scipy.sparse as sp
mp.offline()

In [2]:
v, f = igl.read_triangle_mesh("data/bunny.off")
mp.plot(v, f, shading={"wireframe": True},filename = "Bunny")

Plot saved to file Bunny.html.


# Vertex normal

In [3]:
#Standard face normal
f_normals = np.zeros((v.shape[0],3))

for fi in range(f.shape[0]):
    # get the vector e1, e2
    a = v[f[fi,0]]
    b = v[f[fi,1]]
    c = v[f[fi,2]]
    e1 = b-a
    e2 = c-a
    face_normal = np.cross(e1,e2)
    face_normal = face_normal/np.linalg.norm(face_normal)
    for j in range(3):
        f_normals[f[fi,j],:] += face_normal
norm = np.linalg.norm(f_normals,axis=1)
f_normals[:,0] /= norm
f_normals[:,1] /= norm
f_normals[:,2] /= norm
mp.plot(v,f, n=f_normals, shading={"flat": False},filename = "standard_normal")

Plot saved to file standard_normal.html.


In [4]:
#Area-weighted face normal
a_normals = np.zeros((v.shape[0],3))

for fi in range(f.shape[0]):
    # get the vector e1, e2
    a = v[f[fi,0]]
    b = v[f[fi,1]]
    c = v[f[fi,2]]
    e1 = b-a
    e2 = c-a
    face_normal = np.cross(e1,e2)/2
    for j in range(3):
        a_normals[f[fi,j],:] += face_normal
norm = np.linalg.norm(a_normals,axis=1)
a_normals[:,0] /= norm
a_normals[:,1] /= norm
a_normals[:,2] /= norm

mp.plot(v,f, n=a_normals, shading={"flat": False},filename = "area_normals")

Plot saved to file area_normals.html.


In [5]:
#Mean-curvature normal
normals = np.zeros((v.shape[0],3))
L = igl.cotmatrix(v,f)
tHn = L@v

norm = np.linalg.norm(tHn,axis = 1)

normals[:,0] = tHn[:,0]/norm
normals[:,1] = tHn[:,1]/norm
normals[:,2] = tHn[:,2]/norm

eps = 0.00005
# align all normal with the face normal direction
for i in range(normals.shape[0]):
    angle = np.arccos(np.dot(normals[i],f_normals[i]) / (np.linalg.norm(normals[i])*np.linalg.norm(f_normals[i])))
    temp_normal = normals[i].copy() * -1
    angle2 = np.arccos(np.dot(temp_normal,f_normals[i]) / (np.linalg.norm(temp_normal)*np.linalg.norm(f_normals[i])))
    if angle > angle2:
        normals[i] = temp_normal

# change normal to face normal if norm is too small
normals[norm < eps,:] = a_normals[norm < eps,:]

mp.plot(v,f, n=normals, shading={"flat": False},filename = "Mean_curvature_normal")

Plot saved to file Mean_curvature_normal.html.


In [6]:
#PCA normal
p_normals = np.zeros((v.shape[0],3))
k = 5
# find kth nearest neighbours
from scipy.spatial import distance
D = distance.squareform(distance.pdist(v))
closest = np.argsort(D, axis=1)
closest_matrix = closest[:,:k]

for vi in range(v.shape[0]):
    dist = np.zeros((v.shape[0],1))
    
    # calculate centroid
    xavg = 0
    yavg = 0
    zavg = 0
    for i in range(k):
        xavg += v[closest_matrix[vi,i],0]
        yavg += v[closest_matrix[vi,i],1]
        zavg += v[closest_matrix[vi,i],2]
    xavg/=k
    yavg/=k
    zavg/=k
    m = np.array((xavg,yavg,zavg))
    Y = v[closest_matrix[vi]]-m
    cov = Y.T @ Y
    lam,vs = np.linalg.eig(cov)
    min_lam = np.argmin(lam)
    p_normals[vi] = vs[min_lam]

# align all normal with the face normal direction
x = 0
y = 0
for i in range(p_normals.shape[0]):
    angle = np.arccos(np.dot(p_normals[i],f_normals[i]) / (np.linalg.norm(p_normals[i])*np.linalg.norm(f_normals[i])))
    temp_normal = p_normals[i].copy() * -1
    angle2 = np.arccos(np.dot(temp_normal,f_normals[i]) / (np.linalg.norm(temp_normal)*np.linalg.norm(f_normals[i])))
    if angle > angle2:
        p_normals[i] = temp_normal
        x += 1
    else:
        y+=1

mp.plot(v,f, n=p_normals, shading={"flat": False},filename = "PCA_normal")

Plot saved to file PCA_normal.html.


In [7]:
#Quadratic fitting normal


# Curvature

In [8]:
#gaussian curvature
k = igl.gaussian_curvature(v, f)
print(k)

# colour by curvature
ind = np.zeros((v.shape[0]))
for ki in range(k.shape[0]):
    if k[ki] >= 0:
        ind[ki]=1
    else:
        ind[ki]=2
p = mp.plot(v, f, c=ind, shading={"flat": True},filename = "gaussian")

[-0.02920445  0.11949715  0.02537773 ...  0.01178756  0.27662998
 -0.00310689]
Plot saved to file gaussian.html.


In [9]:
# principal curvature
v1, v2, k1, k2 = igl.principal_curvature(v, f)
h2 = 0.5 * (k1 + k2)
p = mp.plot(v, f, h2, shading={"wireframe": False},filename="principle")

avg = igl.avg_edge_length(v, f) / 2.0
p.add_lines(v + v1 * avg, v - v1 * avg, shading={"line_color": "red"})
p.add_lines(v + v2 * avg, v - v2 * avg, shading={"line_color": "green"})

Plot saved to file principle.html.


# Smoothing with the Laplacian

In [10]:
from scipy.sparse.linalg import spsolve
import scipy.sparse as sp

In [11]:
# Explicit laplacian
iterations = 10000
lam = 0.001
v, f = igl.read_triangle_mesh("data/cow.off")
l = igl.cotmatrix(v, f)
n = igl.per_vertex_normals(v, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)
l = igl.cotmatrix(v, f)
m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
#m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)

# for loop to control the number of iterations
for i in range(iterations):
    Minv = sp.diags(1 / m.diagonal())
    L = Minv @ l
    v = v + lam*0.001*L.dot(v) # vn+1 = Vn + lam dt L(v)
    n = igl.per_vertex_normals(v, f)*0.5+0.5
    c = np.linalg.norm(n, axis=1)

p = mp.plot(v, f, c, shading={"wireframe": False},filename="explicit")

Plot saved to file explicit.html.


In [12]:
# implicit laplacian
iterations = 1
lam = 20
v, f = igl.read_triangle_mesh("data/cow.off")
l = igl.cotmatrix(v, f)
# do the 2d grid laplacian
'''lnew = np.zeros(l.shape)
vvr = igl.adjacency_list(f)
for i in range(v.shape[0]):
    #print(vvr[i])
    sum = np.zeros((1,l.shape[1]))
    #print(sum)
    for j in range(len(vvr[i])):
        sum = sum + l[vvr[i][j]] - l[i]
        #print(sum)
    lnew[i] = sum'''
# covert to the uniform laplacian, incredibly inefficient implemtation
'''for i in range(l.shape[0]):
    for j in range(l.shape[1]):
        if l[i,j] != 0 and i != j:
            l[i,j] = 1
    print(i)
print(l)'''
n = igl.per_vertex_normals(v, f)*0.5+0.5
c = np.linalg.norm(n, axis=1)


# for loop to control the number of iterations
for i in range(iterations):
    m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_BARYCENTRIC)
    #m = igl.massmatrix(v, f, igl.MASSMATRIX_TYPE_VORONOI)
    s = (m - lam*0.001 * l)# results in L
    v = spsolve(s, m.dot(v))
    n = igl.per_vertex_normals(v, f)*0.5+0.5
    c = np.linalg.norm(n, axis=1)


p = mp.plot(v, f, c, shading={"wireframe": False},filename="implicit")

Plot saved to file implicit.html.
